# Prezentacja 3

In [70]:
from pandas import read_csv, to_datetime, read_excel, Series, date_range
import numpy as np
from scipy.stats import t, norm

###  Notowania giełdowe PEPSICO (PEP) - od 13.04.2015 do 31.03.2025, co tydzień

https://www.nasdaq.com/market-activity/stocks/pep/historical?page=1&rows_per_page=10&timeline=y10

In [71]:
dane_notowania = read_csv('dane\\notowania.csv')
dane_notowania['Date'] = to_datetime(dane_notowania['Date'], format='%m/%d/%Y')
dane_notowania = dane_notowania.sort_values('Date')
dane_notowania.set_index('Date', inplace=True)
# Ustalenie startowej daty: 13.04.2015
start_date = to_datetime('2015-04-13')
dane_notowania = dane_notowania[dane_notowania.index >= start_date]
dane_notowania = dane_notowania.resample('7D').first()
daty_notowania = dane_notowania.index.tolist()
dane_notowania = dane_notowania["Close/Last"].tolist()
dane_notowania = [float(x.replace('$', '')) for x in dane_notowania]
# aby uniknąć problemu związanemu z brakiem danych w weekendy i dni niehandlowe, bierzemy notowanie z każdego poniedziałku (jeśli nie ma z danego poniedziałku, to z wtorku etc.)

## VaR dla notowań

### Metoda parametryczna

In [82]:
# Obliczanie tygodniowych zmian procentowych
zmiany_procentowe = Series(dane_notowania).pct_change().dropna() * 100
daty_zmian = daty_notowania[1:]  # pierwszy tydzień nie ma zmiany

# Unormowane dane lub oryginalne zmiany procentowe:
data = -np.array(zmiany_procentowe)

# Dopasuj rozkład t: zwraca df, loc, scale
df_fit, loc_fit, scale_fit = t.fit(data)

print("VaR rzędu 95% dla notowań:", t.ppf(0.95, df_fit, loc=loc_fit, scale=scale_fit))
print("VaR rzędu 99% dla notowań:", t.ppf(0.99, df_fit, loc=loc_fit, scale=scale_fit))

VaR rzędu 95% dla notowań: 3.5073387002719807
VaR rzędu 99% dla notowań: 5.875043542907393


### Metoda historyczna (zwykła)

In [83]:
print("Kwantyl empiryczny dla danych:", np.quantile(data, 0.95))
print("Kwantyl empiryczny dla danych:", np.quantile(data, 0.99))

Kwantyl empiryczny dla danych: 3.5473920358945366
Kwantyl empiryczny dla danych: 5.6245881905998685


In [85]:
def weighted_historical_VaR(data, alpha, lambd=0.96):
    n = len(data)
    w_1 = (1 - lambd) / (1 - np.power(lambd, n))
    weights = np.power(lambd, n - np.arange(1, n + 1)) * w_1
    return np.quantile(data, alpha, weights=weights, method="inverted_cdf")

print(weighted_historical_VaR(data, 0.95))
print(weighted_historical_VaR(data, 0.99))

4.501800720288118
5.623368376490512


### VaR obliczony z użyciem filtrowania szeregiem GARCH

In [91]:
from arch import arch_model

# === 1. Dopasowanie modelu GARCH do zwrotów ===
model = arch_model(data, vol='Garch', p=1, q=1, dist='normal')
res = model.fit(disp='off')

mu_hat = res.params['mu']  # Estymowana wartość średnia
sigma_t = res.conditional_volatility  # σₜ dla t = 1,...,n
L = data # Zwroty

# === 2. Oblicz standaryzowane reszty ===
standardized_residuals = (L - mu_hat) / sigma_t

# === 3. Wyznacz kwantyl empiryczny dla *L_i ===
alpha = 0.99  # poziom istotności
q_alpha = np.quantile(standardized_residuals, alpha)

# === 4. Prognoza sigma_{n+1} ===
# GARCH(1,1): σ²_{t+1} = ω + α * ε²_t + β * σ²_t

# Wyciągamy parametry modelu
omega = res.params['omega']
alpha1 = res.params['alpha[1]']
beta1 = res.params['beta[1]']

# Prognoza wariancji
sigma_n1_sq = omega + alpha1 * (L[-1] - mu_hat)**2 + beta1 * sigma_t[-1]**2
sigma_n1 = np.sqrt(sigma_n1_sq)

# === 5. Oblicz VaR ===
VaR_alpha = sigma_n1 * q_alpha + mu_hat

print(f"📉 Value at Risk (VaR) na poziomie {alpha*100}%: {VaR_alpha:.4f}")

📉 Value at Risk (VaR) na poziomie 99.0%: 8.9195
